Задание 2+

Шаг 1. Сделать непараметрические оценки плотности распределения в виде гистограммы и ядерной оценки МСВ.​

Шаг 2. Оценить мат. ожидание и дисперсию МСВ.​

Шаг 3. Сделать непараметрические оценки условных распределений, мат. ожиданий и дисперсий.​

Шаг 4. Оценить парные коэффициенты корреляции, дов. интервалы для них и уровни значимости. ​

Шаг 5. Выбрать постановку задачи для регрессии. Оценить коэффициенты множественной корреляции (предиктанта на предикторы).​

Шаг 6. Построить регрессионную модель. Выполнить анализ мультиколлинеарности и регуляризацию (при необходимости).​

Шаг 7. Выполнить анализ качества предсказательной модели. Анализ распределения остатков, коэффициент детерминации.

In [ ]:
# Импорт пакетов 
import numpy as np
import statsmodels.api as sm
import sklearn
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats
from PIL import Image
import scipy

In [ ]:
# Импорт данных из csv-файла - 
# ежедневные данные о погоде с многочисленных австралийских метеостанций.

path_to_file = 'weatherAUS.csv'
source_df = pd.read_csv(path_to_file, engine='python')

source_df.head(5)

In [ ]:
#Отбираем нужные параметры
new_df = source_df[['Temp3pm','Evaporation','Sunshine','Pressure3pm','WindGustSpeed']]

#Удаление значений Nan
#Можно заполнить средними, если мало пропусков.
new_df = new_df.dropna()

new_df.head(5)

Шаг 1. Сделать непараметрические оценки плотности распределения в виде гистограммы и ядерной оценки МСВ.​

In [ ]:
#Для трёхмерного пространства признаков
pca2 = PCA(n_components=2)
pca1 = PCA(n_components=1)

#Переход к главным компонентам
X_3dim_new = pd.DataFrame(pca2.fit_transform(new_df))
X_3dim_new1 = pd.DataFrame(pca1.fit_transform(new_df))
scaler = StandardScaler()

#Масштабирование главных компонент
X_3dim_new = pd.DataFrame(scaler.fit_transform(X_3dim_new))
X_3dim_new1 = pd.DataFrame(scaler.fit_transform(X_3dim_new1))

#График в пространстве 3-х размерностей исходных данных
xs=new_df[['Temp3pm']]
ys=new_df[['Evaporation']]
zs=new_df[['Sunshine']] 
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(xs, ys, zs)
ax.set_xlabel('Temp3pm')
ax.set_ylabel('Evaporation')
ax.set_zlabel('Sunshine')
ax.set_title('Исходные данные в пространстве трёх выбранных признаков')


Шаг 2. Оценить мат. ожидание и дисперсию МСВ.​

In [ ]:
# Вычисление выборочного среднего, дисперсии
mean = new_df.mean()
var = np.var(new_df)

print('Мат. ожидание \n', mean, '\n')
print('Дисперсия \n', var)

Шаг 3. Сделать непараметрические оценки условных распределений, мат. ожиданий и дисперсий.

Шаг 4. Оценить парные коэффициенты корреляции, дов. интервалы для них и уровни значимости. ​

In [ ]:
#отбор столбцов для построения коррекляционной матрицы
X_train = new_df

#построение диагональной маски
mask = np.zeros_like(X_train.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

f, ax = plt.subplots(figsize=(10, 10))

sns.heatmap(X_train.corr(), mask = mask, annot=True, fmt= '.1f', ax = ax, cmap = 'Blues')

In [ ]:
#Визуальзация зависимости признаков между собой
X = new_df[['Temp3pm','Sunshine','Pressure3pm','WindGustSpeed']]
scaler = StandardScaler()
y = new_df[['Evaporation']]
X = pd.DataFrame(scaler.fit_transform(X))
X.columns = ['Temp3pm','Sunshine','Pressure3pm','WindGustSpeed']


#Создание subplot - нескольких графиков на одной картинке
fig, axs = plt.subplots(4, 1)
plt.subplots_adjust(wspace=0, hspace=2.5)
axs[0].scatter(X[['Temp3pm']],y,s=10)
axs[0].set_xlabel('Temp3pm')
axs[0].set_ylabel('y')
axs[0].grid(True)

axs[1].scatter(X[['Sunshine']],y,s=10)
axs[1].set_xlabel('Sunshine')
axs[1].set_ylabel('y')
axs[1].grid(True)

axs[2].scatter(X[['Pressure3pm']],y,s=10)
axs[2].set_xlabel('Pressure3pm')
axs[2].set_ylabel('y')
axs[2].grid(True)

axs[3].scatter(X[['WindGustSpeed']],y,s=10)
axs[3].set_xlabel('WindGustSpeed')
axs[3].set_ylabel('y')
axs[3].grid(True)

Шаг 5. Выбрать постановку задачи для регрессии. Оценить коэффициенты множественной корреляции (предиктанта на предикторы).

In [ ]:
#Отбор трёх самых значимых признаков
X = new_df[['Temp3pm','Sunshine','Pressure3pm','WindGustSpeed']]
#Нормирование признаков
scaler = StandardScaler()
y = new_df[['Evaporation']]
X = pd.DataFrame(scaler.fit_transform(X))
X.columns = ['Temp3pm','Sunshine','Pressure3pm','WindGustSpeed']
scaler = StandardScaler()
y = pd.DataFrame(scaler.fit_transform(y))
y.columns = ['Evaporation']
#Деление на тестовую и тренировочную выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
#Создание и обучение модели линейной регрессии
reg = LinearRegression(normalize=True)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
y_pred_all = np.array(reg.predict(X))

Шаг 6. Построить регрессионную модель. Выполнить анализ мультиколлинеарности и регуляризацию (при необходимости).​

In [ ]:
#Вычисление метрик модели
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print('Mean absolute error = ', mae)
print('Mean squared error = ', mse)
# mape = mean_absolute_percentage_error(y_test, y_pred)
# print('Mean absolute percentage error = ', mape)


Шаг 7. Выполнить анализ качества предсказательной модели. Анализ распределения остатков, коэффициент детерминации.

In [ ]:
#Построение квантильного биплота по реальным и предсказанным значениям
percs = np.linspace(0, 100, 21)
qn_first = np.percentile(y, percs)
qn_second = np.percentile(y_pred_all, percs)
plt.figure(figsize=(6, 6))

min_qn = np.min([qn_first.min(), qn_second.min()])
max_qn = np.max([qn_first.max(), qn_second.max()])
x = np.linspace(min_qn, max_qn)

plt.plot(qn_first, qn_second, ls="", marker="o", markersize=6)
plt.plot(x, x, color="k", ls="--")

plt.xlabel(u'Реальные значения')
plt.ylabel(u'Предсказанные значения')

In [ ]:
x = []

for i in range(len(y_test)):
    x.append(i)
#График реальных и предсказанных значений
plt.scatter(x, y_test, label = u'Реальное значение')
plt.scatter(x, y_pred, label = u'Предсказанное линейной моделью')
plt.title(u'Количество испарений')
plt.legend(loc="center right",borderaxespad=0.1, bbox_to_anchor=(1.7, 0.5))

In [ ]:
#Построение распределения остатков
y1 = np.array(y)
y2 = np.array(y_pred_all)
y_diff = y1[:,0] - y2[:,0]
sns.distplot(y_diff, kde=False)

In [ ]:
# Вычисление коэффициента детерминации.
mod = sm.OLS(y_train, X_train)
res = mod.fit()
print (res.conf_int(0.01))  